In [24]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import seaborn as sns

# Data

I performed an open ionbot and msfragger search (oxididation of M variable, max 2 missed cleavages).

See SOP document for more details on how MSFragger was ran.

In [25]:
#set this your local folder as the data was pushed to the repo as well
folder = "/home/compomics/tools/ionbot_paper"
datasets = ["PXD001468","PXD000561","PXD000815","PXD001250","TMTCPTAC"]
#datasets = ["PXD001468"]


 # Results
 
Load ionbot PSM results (first-matches + coeluting matches + features) + protein results.

In [26]:
def contains_missed_cleavage(x):
    if "K" in x[:-1]:
        return True
    if "R" in x[:-1]:
        return True
    return False

def is_in_s(x):
    return x in s

In [27]:
results = {}
f = open("chimeric_size.csv","w")
f.write("dataset,#matches,count,type\n")
for datas in datasets:
    for suffix in ["","_nort","_nocorr","_nocorr_nort","_closed"]:
    #for suffix in [""]:
        dataset = datas + suffix
        print(dataset)
        #Processing dataset
        if "PXD001468" in dataset:
            def get_scan_ionbot(x):
                return int(x.split('.')[1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        else:
            def get_scan_ionbot(x):
                print(x)
                return int(x.split('=')[-1])
            def get_spectrum_file_ionbot(x):
                return x.split('.')[0]
        folder2 = folder + "/" + dataset
        ionbot = pd.read_csv("%s/ionbot.first.csv"%folder2)
        ionbot["rank"] = ["first"]*len(ionbot)
        tmp = pd.read_csv("%s/ionbot.lower.csv"%folder2)
        tmp["rank"] = ["lower"]*len(tmp)
        ionbot = pd.concat([ionbot,tmp])
        tmp = pd.read_csv("%s/ionbot.features.csv"%folder2)
        ionbot = ionbot.merge(tmp,on="ionbot_match_id",how="left")
        if "PXD001468" in dataset:
            ionbot["scan"] = ionbot["spectrum_title"].apply(get_scan_ionbot)
            ionbot["sfile"] = ionbot["spectrum_title"].apply(get_spectrum_file_ionbot)
        else:
            ionbot["sfile"] = ionbot["spectrum_file"].apply(get_spectrum_file_ionbot)
        ionbot["PSM"] = ionbot["scan"].astype(str) + ionbot["sfile"] +  "XYX" +  ionbot["matched_peptide"]
        ionbot = ionbot[(ionbot["database"]=="T")&(ionbot["q-value"]<=0.01)]
        ionbot["num_passed"] = ionbot.groupby(["sfile","scan"])["scan"].transform("count")

        print(ionbot["num_passed"].value_counts())
        print(ionbot["rank"].value_counts())
        if suffix == "":
            tmp = ionbot["num_passed"].value_counts()
            for i in range(1,len(tmp)):
                f.write("%s,%i,%i,""\n"%(dataset,i+1,tmp.iloc[i]))        
        if suffix == "_nocorr_nort":
            tmp = ionbot["num_passed"].value_counts()
            for i in range(1,len(tmp)):
                f.write("%s,%i,%i,_nocorr_nort\n"%(dataset,i+1,tmp.iloc[i]))        
        results[dataset] = ionbot
f.close()


PXD001468
1    679545
2     52248
3      2469
4        72
Name: num_passed, dtype: int64
first    706510
lower     27824
Name: rank, dtype: int64
706510
PXD001468_nort
1    654467
2     48248
3      2130
4        56
Name: num_passed, dtype: int64
first    679315
lower     25586
Name: rank, dtype: int64
679315
PXD001468_nocorr
1    632580
2     39592
3      1536
4        32
Name: num_passed, dtype: int64
first    652896
lower     20844
Name: rank, dtype: int64
652896
PXD001468_nocorr_nort
1    596038
2     34462
3      1194
4        20
Name: num_passed, dtype: int64
first    613672
lower     18042
Name: rank, dtype: int64
613672
PXD001468_closed
1    457149
2      5382
3       129
4         4
Name: num_passed, dtype: int64
first    459884
lower      2780
Name: rank, dtype: int64
459884
PXD000561
1    259283
2     22700
3      1446
4        48
Name: num_passed, dtype: int64
first    271127
lower     12350
Name: rank, dtype: int64
271127
PXD000561_nort


/home/compomics/.local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1    253992
2     21408
3      1278
4        40
Name: num_passed, dtype: int64
first    265132
lower     11586
Name: rank, dtype: int64
265132
PXD000561_nocorr
1    247033
2     18084
3       915
4        28
Name: num_passed, dtype: int64
first    256387
lower      9673
Name: rank, dtype: int64
256387
PXD000561_nocorr_nort
1    236028
2     15768
3       669
4        20
Name: num_passed, dtype: int64
first    244140
lower      8345
Name: rank, dtype: int64
244140
PXD000561_closed
1    201499
2      4264
3       105
4         8
Name: num_passed, dtype: int64
first    203668
lower      2208
Name: rank, dtype: int64
203668
PXD000815
1    83362
2     3760
3      120
Name: num_passed, dtype: int64
first    85282
lower     1960
Name: rank, dtype: int64
85282
PXD000815_nort
1    78272
2     3410
3      105
Name: num_passed, dtype: int64
first    80012
lower     1775
Name: rank, dtype: int64
80012
PXD000815_nocorr
1    78581
2     3150
3       75
4        4
Name: num_passed, dtype: int64
first

In [28]:
print(ionbot["num_passed"].value_counts())
tmp = ionbot["num_passed"].value_counts()
for i in range(1,len(tmp)):
    print(tmp.iloc[i])

1    249137
2      1830
3         9
Name: num_passed, dtype: int64
1830
9


In [29]:
def is_in_s(x):
    return x in s

to_write = []

for datas in datasets:
    dataset = datas
    #compute statistics
    print(dataset)

    dataset = datas + "_closed"
    ionbot = results[dataset]
    full = len(ionbot)
    print(full)
    to_write.append(["","count","PSM",datas,"_closed",full])
    full = len(ionbot["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_closed",full])

    #compute group FDR

    dataset = datas
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    #ionbot[ionbot["is_7Da"]==True].sort_values("psm_score").to_csv("wide_error_pxd001468.csv",index=False)
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open",unexp])

    dataset = datas + "_nort"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_noR",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_noR",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_noR",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_noR",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_noR",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_noR",unexp])

    dataset = datas + "_nocorr"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_no_corr",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_no_corr",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_no_corr",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_no_corr",unexp])

    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","count","PSM",datas,"_open_noR_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","PSM",datas,"_open_noR_no_corr",wide])
    to_write.append(["is_unexpected","count","PSM",datas,"_open_noR_no_corr",unexp])
    full = len(ionbot[ionbot["rank"]=="first"]["matched_peptide"].unique())
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)]["matched_peptide"].unique())
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())]["matched_peptide"].unique())
    to_write.append(["","count","peptide",datas,"_open_noR_no_corr",full-wide-unexp])
    to_write.append(["is_7Da","count","peptide",datas,"_open_noR_no_corr",wide])
    to_write.append(["is_unexpected","count","peptide",datas,"_open_noR_no_corr",unexp])

    #overlap between open and open without corr and rt information
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]
    s2 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"_open",len(s1.difference(s2))])
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"intersection",len(s1.intersection(s2))])
    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)
    #now I cut the peptide bacause I want the overlap in title
    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))
    to_write.append(["","overlap_open_noR_no_corr","PSM",datas,"_open_noR_no_corr",uniques2-overruled])
    to_write.append(["overruled","overlap_open_noR_no_corr","PSM",datas,"_open_noR_no_corr",overruled])

    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["matched_peptide"])
    dataset = datas + "_nocorr_nort"
    ionbot = results[dataset]
    s2 = set(ionbot["matched_peptide"])
    to_write.append(["","overlap_open_noR_no_corr","peptide",datas,"_open",len(s1.difference(s2))])
    to_write.append(["","overlap_open_noR_no_corr","peptide",datas,"intersection",len(s1.intersection(s2))])
    to_write.append(["","overlap_open_noR_no_corr","peptide",datas,"_open_noR_no_corr",len(s2.difference(s1))])


    #overlap between open and closed
    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["PSM"])
    dataset = datas + "_closed"
    ionbot = results[dataset]
    s2 = set(ionbot["PSM"])
    to_write.append(["","overlap_closed","PSM",datas,"intersection",len(s1.intersection(s2))])

    uniques2 = len(s2.difference(s1))
    tmp = s1.difference(s2)
    #now I cut the peptide bacause I want the overlap in title
    tmp2 = []
    for s in tmp:
        tmp2.append(s.split("XYX")[0])
    tmp2 = set(tmp2)
    tmp = s2.difference(s1)

    tmp3 = []
    for s in tmp:
        tmp3.append(s.split("XYX")[0])
    tmp3 = set(tmp3)
    overruled = len(tmp2.intersection(tmp3))
    to_write.append(["","overlap_closed","PSM",datas,"_closed",uniques2-overruled])
    to_write.append(["overruled","overlap_closed","PSM",datas,"_closed",overruled])

    dataset = datas
    ionbot = results[dataset]
    s = s1.difference(s2)
    ionbot["unique"] = ionbot["PSM"].apply(is_in_s)
    ionbot = ionbot[ionbot["unique"]==True]
    ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
    ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))
    full = len(ionbot[ionbot["rank"]=="first"])
    wide = len(ionbot[(ionbot["rank"]=="first")&(ionbot["is_7Da"]==True)])
    unexp = len(ionbot[(ionbot["rank"]=="first")&(ionbot["unexpected_modification"].notna())])
    to_write.append(["","overlap_closed","PSM",datas,"_open",full-wide-unexp])
    to_write.append(["is_7Da","overlap_closed","PSM",datas,"_open",wide])
    to_write.append(["is_unexpected","overlap_closed","PSM",datas,"_open",unexp])

    dataset = datas
    ionbot = results[dataset]
    s1 = set(ionbot[ionbot["rank"]=="first"]["matched_peptide"])
    dataset = datas + "_closed"
    ionbot = results[dataset]
    s2 = set(ionbot["matched_peptide"])
    to_write.append(["","overlap_closed","peptide",datas,"_open",len(s1.difference(s2))])
    to_write.append(["","overlap_closed","peptide",datas,"intersection",len(s1.intersection(s2))])
    to_write.append(["","overlap_closed","peptide",datas,"_closed",len(s2.difference(s1))])

to_write.append(["","count","PSM","PXD001468","_open_om",586403])
to_write.append(["","count","PSM","PXD000561","_open_om",227746])
to_write.append(["","count","PSM","PXD001250","_open_om",353869])
to_write.append(["","count","PSM","PXD000815","_open_om",73342])
to_write.append(["","count","PSM","TMTCPTAC","_open_om",367960])
to_write.append(["","count","peptide","PXD001468","_open_om",130485])
to_write.append(["","count","peptide","PXD000561","_open_om",63683])
to_write.append(["","count","peptide","PXD001250","_open_om",82164])
to_write.append(["","count","peptide","PXD000815","_open_om",21853])
to_write.append(["","count","peptide","TMTCPTAC","_open_om",149891])




PXD001468
462664


/tmp/ipykernel_3776874/3389331350.py:158: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ionbot["massdiff"] = np.abs(ionbot["precursor_mass"]-ionbot["peptide_mass"])
/tmp/ipykernel_3776874/3389331350.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ionbot["is_7Da"] = ((ionbot["unexpected_modification"].isna()) & (ionbot["massdiff"]>0.2))


PXD000561
205876
PXD000815
75154
PXD001250
245537
TMTCPTAC
250976


In [30]:
to_write = pd.DataFrame(to_write,columns=["unexpected","type","level","spectrum_file","search","value"])
to_write.to_csv("id_overlap.csv",index=False)

In [31]:
to_write

,unexpected,type,level,spectrum_file,search,value
0,,count,PSM,PXD001468,_closed,462664
1,,count,peptide,PXD001468,_closed,123006
2,,count,PSM,PXD001468,_open,411971
3,is_7Da,count,PSM,PXD001468,_open,156443
4,is_unexpected,count,PSM,PXD001468,_open,138096
...,...,...,...,...,...,...
215,,count,peptide,PXD001468,_open_om,130485
216,,count,peptide,PXD000561,_open_om,63683
217,,count,peptide,PXD001250,_open_om,82164
218,,count,peptide,PXD000815,_open_om,21853


In [32]:
for dataset in datasets:
    print(dataset)
    tmp = to_write[(to_write["spectrum_file"]==dataset)&(to_write["level"]=="PSM")]
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_closed")]["value"].sum()
    v2 = tmp[(tmp["type"]=="overlap_closed")&(tmp["search"]=="_closed")]["value"].sum()
    v = v2 / (v1+v2)
    print("lost in closed: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with RT: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr: %f"%(100*v))
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr-RT: %f"%(100*v))
    tmp = to_write[(to_write["spectrum_file"]==dataset)&(to_write["level"]=="peptide")]
    v1 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open")]["value"].sum()
    v2 = tmp[(tmp["type"]=="count")&(tmp["search"]=="_open_noR_no_corr")]["value"].sum()
    v = (v1 - v2) / v1
    print("gained with corr-RT: %f"%(100*v))



PXD001468
lost in closed: 11.765458
gained with RT: 3.849202
gained with corr: 7.588569
gained with corr-RT: 13.140366
gained with corr-RT: 10.905988
PXD000561
lost in closed: 11.263405
gained with RT: 2.211141
gained with corr: 5.436567
gained with corr-RT: 9.953638
gained with corr-RT: 9.142201
PXD000815
lost in closed: 18.712887
gained with RT: 6.179499
gained with corr: 5.980160
gained with corr-RT: 13.571445
gained with corr-RT: 10.784493
PXD001250
lost in closed: 13.649424
gained with RT: 2.870190
gained with corr: 6.811778
gained with corr-RT: 11.831488
gained with corr-RT: 11.685840
TMTCPTAC
lost in closed: 6.744399
gained with RT: 0.853710
gained with corr: 10.479330
gained with corr-RT: 12.716640
gained with corr-RT: 11.814007
